In [ ]:
#%% md

# Sprint21 自然言語処理

#%% md

# データの準備

下記のURLから、圧縮ファイルをダウンロードしてください。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

ダウンロードした圧縮ファイルを解凍し、このsprint21.ipynbと同じ階層においてください。

#%% md

# ライブラリのimport

#%%

from sklearn.datasets import load_files
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import lightgbm as lgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from gensim.models import word2vec
import numpy as np
from sklearn.preprocessing import normalize
import re

#%% md

# データの読み込み

#%%

train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target
test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target

#%%

# テスト出力
print("x : {}".format(x_train[0]))
print(np.array(x_train).shape,np.array(x_test).shape,np.array(y_train).shape,np.array(y_test).shape)

#%% md

# 問題1　BoWのスクラッチ実装

まずは、sklearnでBoWを計算してみます。

#%%

# 仮のデータ
mini_dataset = ['This movie is very good.','This film is a good','Very bad. Very, very bad.']

# インスタンス化
vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
# 実行
bow = (vectorizer.fit_transform(mini_dataset)).toarray()
# DataFrame化
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
# 出力
display(df)

#%% md

次に、スクラッチで作ってみます。

#%%

def bow(data):
    """BoW算出
    Parameters
    -----------
    data : 文章リスト
    """
    ## 単語リスト作成
    # 小文字に統一
    # !除去
    # 文字列を半角スペース基準で分割し、リスト化
    row_data = [i.lower().replace('!', '').split(' ') for i in data]
    # 1次元のリストに(単語リスト)
    feature_names = set(list(itertools.chain.from_iterable(row_data)))
    
    ## bow計算
    bow = []
    # 1つづつ文章でループ
    for index,row in enumerate(data):
        bow.append([])
        # 単語リストでループ
        for feature_name in feature_names:
            # 何個含まれているか
            num = row_data[index].count(feature_name)
            # 追加
            bow[index].append(num)
    return feature_names,bow

# 仮データの定義
mini_dataset = ['This movie is SOOOO funny!!!','What a movie! I never','best movie ever!!!!! this movie']
# bow関数実行
feature_names,bow = bow(mini_dataset)
# DF化
df = pd.DataFrame(bow, columns=feature_names)
# 出力
display(df)

#%% md

# 問題2　TF-IDFの計算

#%%

# nltkライブラリのstopwordsを利用
stop_words = nltk.download('stopwords')
stop_words = stopwords.words('english')
print("stop word : {}".format(stop_words))

#%%

# tfidfの算出
vectorizer = TfidfVectorizer(stop_words= stop_words, max_features=5000)
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.fit_transform(x_test)

#%%

# テスト出力
print(X_train.shape, X_test.shape)

#%% md

# 問題3　TF-IDFを用いた学習

#%%

# lightGBMを用いた学習
lgb = lgb.LGBMClassifier().fit(X_train,y_train)
# 推定
y_pred = lgb.predict(X_test)

#%%

# 結果出力
print("{}".format(lgb.score(X_test, y_test)))
print(confusion_matrix(y_test, y_pred))

#%% md

# 問題4　TF-IDFのスクラッチ実装

まずは、sklearnでtfidfを計算してみます。

#%%

# 仮データ
mini_dataset = ['This movie is SOOOO funny!!!','What a movie! I never','best movie ever!!!!! this movie']

#%%

# インスタンス化
tfidf_model = TfidfVectorizer()
# 計算
tfidf = tfidf_model.fit_transform(mini_dataset)
# DF化
tfidf = pd.DataFrame(tfidf.toarray(), columns=tfidf_model.get_feature_names())
# 出力
tfidf

#%% md

次に、スクラッチで作ってみます。

#%%

# インスタンス化
cv_model = CountVectorizer()
# 計算
cv= cv_model.fit_transform(mini_dataset)
# 扱いやすいように配列化
cv_array = cv.toarray()

# TF値計算
N = cv_array.shape[0]
tf = np.array([cv_array[i, :] / np.sum(cv_array, axis=1)[i] for i in range(N)])

# IDF値計算
df = np.count_nonzero(cv_array, axis=0)
idf = np.log((1 + N) / (1 + df)) + 1

# normalize
tfidf = normalize(tf*idf)
tfidf = pd.DataFrame(tfidf, columns=cv_model.get_feature_names())
tfidf

#%% md

# 問題5　コーパスの前処理

この問題以降は簡単のため、1文のみを扱う。

#%%

# 簡単のため、URL含んでそうな1文抜き出す
with_url = 0
for i, s in enumerate(x_train):
    if 'www' in s:
        with_url = i
        print('-----before processing')
        print(s)
        break
no_preprocessing = x_train[with_url]

# urlは除外
after_preprocessing1 = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", no_preprocessing) 
# タグ除去
after_preprocessing2 = re.sub(r'<[^>]+>', " ", after_preprocessing1) 
# 数字と英字以外除去
after_preprocessing3 = re.sub(r"[^0-9a-zA-Z ]", "", after_preprocessing2) 
# 小文字に統一
after_preprocessing = after_preprocessing3.lower() 

print('-----after processing')
print(after_preprocessing)

#%% md

# 問題6　Word2Vecの学習

#%%

# 単語リスト
word_list = [after_preprocessing.split(' ')]

# vector_size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# epochs: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
model = word2vec.Word2Vec(word_list,min_count=1) 

#%%

# 確認
model.wv['hand']
